<a href="https://colab.research.google.com/github/aygul0790/Bootcamp/blob/main/Introduction_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

An Agent is a system that leverages an AI model to interact with its environment to achieve a user-defined objective. It combines reasoning, planning, and action execution (often via external tools) to fulfil tasks.

LlamaIndex supports three main types of reasoning agents:


Advanced Custom Agents - These use more complex methods to deal with more complex tasks and workflows.

1.   Function Calling Agents - These work with AI models that can call specific functions.
2.   ReAct Agents - These can work with any AI that does chat or text endpoint and deal with complex reasoning tasks.
3. Advanced Custom Agents - These use more complex methods to deal with more complex tasks and workflows.


# Introduction to Agents with LlamaIndex

This notebook provides a practical introduction to building AI agents using LlamaIndex and OpenAI. We'll create two independent agents:
1. A Recipe Assistant that helps users find recipes and nutritional information
2. A Financial Assistant that retrieves stock data and performs calculations

## 1. Setting Up the Environment

First, let's install the necessary packages and set up our environment.

In [ ]:
# Install the required packages
!pip install llama-index llama-index-llms-openai llama-index-tools-yahoo-finance openai -q

import os
import asyncio
from openai import OpenAI
from llama_index.llms.openai import OpenAI as LlamaOpenAI
from llama_index.core.agent.workflow import AgentWorkflow
from IPython.display import display, Markdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.4 MB/s eta 0:00:00


Next, we'll set up our OpenAI API key. This is required to use OpenAI's language models.

In [ ]:
# To use your own OpenAI API key, uncomment and add your key here
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Or you can input it securely using getpass
from getpass import getpass
api_key = getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = api_key

Enter your OpenAI API Key: ··········


## 2. Recipe Assistant Agent

### 2.1 Setting Up the Recipe Database

We'll create a mock recipe database to demonstrate how an agent can interact with structured data.

In [ ]:
# Mock recipe database
RECIPES = {
    "banana smoothie": {
        "ingredients": ["banana", "milk", "honey", "ice"],
        "nutrition": {"calories": 210, "protein": "6g", "fat": "2g"},
        "type": "vegetarian"
    },
    "avocado toast": {
        "ingredients": ["bread", "avocado", "lemon juice", "salt", "pepper"],
        "nutrition": {"calories": 300, "protein": "5g", "fat": "20g"},
        "type": "vegetarian"
    },
    "pasta primavera": {
        "ingredients": ["pasta", "zucchini", "bell pepper", "tomato", "olive oil", "garlic"],
        "nutrition": {"calories": 450, "protein": "12g", "fat": "15g"},
        "type": "vegetarian"
    },
    "chicken curry": {
        "ingredients": ["chicken", "curry powder", "coconut milk", "onion", "garlic", "rice"],
        "nutrition": {"calories": 520, "protein": "35g", "fat": "22g"},
        "type": "non-vegetarian"
    }
}

### 2.2 Defining Tool Functions for the Recipe Agent

Tool functions are the building blocks of our agent's capabilities. Each function represents a specific task the agent can perform.

#### What are Tool Functions?

Tool functions are specialized pieces of code that an agent can call to perform specific tasks. They:
- Have well-defined inputs and outputs
- Perform one specific function (following the single responsibility principle)
- Are designed to be used by the agent, not directly by users
- Have clear docstrings that help the agent understand when to use them

#### Our Recipe Agent Tools

We'll create four tool functions for our recipe agent:

1. **`find_recipe_by_ingredients`**:
   - **Purpose**: Searches for recipes containing specific ingredients
   - **Input**: A list of ingredient strings
   - **Output**: A list of recipe names that include all the specified ingredients
   - **Use case**: When a user asks "What can I make with X and Y?"

2. **`get_ingredients`**:
   - **Purpose**: Retrieves all ingredients for a specific recipe
   - **Input**: A recipe name
   - **Output**: A list of ingredients
   - **Use case**: When a user asks "What do I need for X recipe?"

3. **`get_nutrition_info`**:
   - **Purpose**: Provides nutritional information for a recipe
   - **Input**: A recipe name
   - **Output**: A dictionary of nutritional facts (calories, protein, fat)
   - **Use case**: When a user asks "How many calories are in X?"

4. **`get_recipe_type`**:
   - **Purpose**: Determines if a recipe is vegetarian or non-vegetarian
   - **Input**: A recipe name
   - **Output**: A string indicating the recipe type
   - **Use case**: When a user asks "Is X vegetarian?"

These functions create a clear separation between the agent's reasoning (handled by the LLM) and the data access (handled by the tools). This separation is a key design principle in agent development.

In [ ]:
# Define tool functions for our recipe assistant
def find_recipe_by_ingredients(ingredients: list[str]) -> list[str]:
    """Find recipes that contain all the given ingredients."""
    return [name for name, data in RECIPES.items()
            if all(ing in data["ingredients"] for ing in ingredients)]

def get_ingredients(recipe_name: str) -> list[str]:
    """Return ingredients for a given recipe."""
    return RECIPES.get(recipe_name.lower(), {}).get("ingredients", [])

def get_nutrition_info(recipe_name: str) -> dict:
    """Return nutritional information for a given recipe."""
    return RECIPES.get(recipe_name.lower(), {}).get("nutrition", {})

def get_recipe_type(recipe_name: str) -> str:
    """Return whether a recipe is vegetarian or non-vegetarian."""
    return RECIPES.get(recipe_name.lower(), {}).get("type", "unknown")

### 2.3 Creating the Recipe Agent Workflow

Now we'll create the agent workflow by combining our tool functions with a language model. The system prompt defines the agent's personality and purpose.

#### What is an Agent Workflow?

An agent workflow connects large language models (LLMs) with tools to create an intelligent system that can:
- Understand natural language requests
- Determine which tools are needed to fulfill requests
- Call the appropriate tools with the right parameters
- Format the results in a human-friendly way

#### Components of Our Recipe Agent

Our agent workflow consists of three key components:

1. **Tool Functions**:
   - The four specialized functions we defined earlier
   - They give the agent the ability to search recipes, retrieve ingredients, etc.
   - Without these tools, the agent would have no way to access our recipe data

2. **Language Model (LLM)**:
   - We're using OpenAI's `gpt-4o-mini` model via LlamaIndex's wrapper
   - The LLM acts as the "brain" of our agent
   - It interprets user questions and decides which tools to use
   - It also formats the final responses in natural language

3. **System Prompt**:
   - This is essentially the "personality" and instructions for our agent
   - It tells the LLM to act as a cooking assistant
   - It guides the agent to be helpful about recipes, ingredients, and nutrition
   - A well-crafted system prompt is critical for agent behavior

#### The AgentWorkflow Class

LlamaIndex's `AgentWorkflow.from_tools_or_functions()` method:
- Takes our tool functions and makes them available to the LLM
- Configures the agent with our system prompt
- Creates a streamlined interface where we simply pass user messages
- Handles all the complex orchestration between the LLM and tools



In [ ]:
# Create the LLM and recipe agent workflow
llm = LlamaOpenAI(model="gpt-4o-mini")

recipe_workflow = AgentWorkflow.from_tools_or_functions(
    [find_recipe_by_ingredients, get_ingredients, get_nutrition_info, get_recipe_type],
    llm=llm,
    system_prompt="You are a helpful cooking assistant. Help users find recipes, list ingredients, and provide nutritional info."
)

### 2.4 Interacting with the Recipe Agent

We'll create a function to query our recipe agent and display the results nicely.

In [ ]:
# For demonstration in notebook cells
async def recipe_query(question):
    """Send a single query to the recipe agent and display the response"""
    response = await recipe_workflow.run(user_msg=question)
    display(Markdown(f"**Recipe Assistant:** {response}"))
    return response

#### Understanding Asynchronous Functions in Python

Our interaction function is defined with `async def`, which indicates it's an asynchronous function:

async def recipe_query(question):

#### What is asynchronous programming?

Asynchronous programming allows operations to run in the background while the program continues execution. This is particularly useful for:

- **I/O-bound operations**: Operations that spend time waiting for external resources (like API calls or database queries)
- **Concurrent tasks**: Running multiple operations at the same time
- **Responsive interfaces**: Keeping an application responsive while performing long-running tasks

#### Why use `async` with our agent?

1. **LLM API calls are I/O-bound**: When we call the recipe agent, it makes API requests to the LLM, which can take time
2. **Prevents blocking**: Using `async` prevents these API calls from blocking other operations in the notebook
3. **Google Colab compatibility**: Colab works well with asynchronous code, especially for user interfaces

#### Key components of async Python:

- `async def`: Defines a coroutine function (not a regular function)
- `await`: Pauses execution until the awaited coroutine completes
- `await recipe_workflow.run(...)`: This pauses until the agent returns a response

#### The Recipe Query Function

Our function does several things:

1. Takes a user question as input
2. Sends it to the agent workflow using `await`
3. Receives the response from the agent
4. Formats the response using Markdown for nicer display
5. Returns the raw response for potential further processing

This simple interface hides the complexity of what's happening behind the scenes, where the agent is:
- Analyzing the question
- Deciding which tool to use
- Calling the appropriate tool
- Formatting a helpful response

When we call this function with different types of recipe questions, we'll see how the agent adapts its behavior based on what we're asking.

### 2.5 Adding Conversation Memory to the Recipe Agent

For a more natural interaction, we'll add conversation memory to allow multi-turn conversations. This is a crucial component of any practical agent system.

#### What is Conversation Memory?

Conversation memory allows an agent to "remember" previous exchanges in a conversation, enabling it to:

* **Maintain context**: Understand references to previously mentioned topics
* **Follow conversation flow**: Create a more natural dialogue experience
* **Handle follow-up questions**: Process questions like "What about the pasta recipe?" that rely on previous context
* **Build upon previous responses**: Avoid repeating information already provided

#### How It Works

In our implementation:

1. We create a simple list (`recipe_conversation`) that stores the entire conversation history
2. Each user input and agent response is added to this list as it occurs
3. When the user asks a new question, we send the entire conversation history as context
4. The language model can then reference this history when determining how to respond

Without conversation memory, each interaction would be treated as an isolated exchange, forcing users to provide complete context with every question. This would result in a rigid, unnatural experience.

The example below demonstrates a simple conversation memory implementation using a list to track the entire conversation. In production systems, more sophisticated approaches might be used, such as:

* Summarizing long conversations to stay within token limits
* Maintaining entity and state trackers
* Using vector databases to store relevant conversation fragments
* Implementing forgetting mechanisms for very long conversations

In [ ]:
# Create a conversation memory for the recipe agent
recipe_conversation = []

async def recipe_chat_with_memory():
    """Interactive chat with the recipe agent that maintains conversation context"""
    print("👩‍🍳 Recipe Assistant Chat! Type 'exit' to quit.")

    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            print("👋 Goodbye!")
            break

        # Add to conversation context
        recipe_conversation.append(f"User: {user_input}")
        context = "\n".join(recipe_conversation)

        try:
            # Get response
            response = await recipe_workflow.run(user_msg=context)
            print(f"👩‍🍳 Recipe Assistant: {response}\n")

            # Add to conversation context
            recipe_conversation.append(f"Assistant: {response}")
        except Exception as e:
            print(f"Error: {str(e)}")



### Memory management

Memory management is a crucial part of conversational agent design. While we want our agent to remember relevant context, we also need mechanisms to forget or reset when appropriate. In more complex systems, this might involve sophisticated memory management rather than simply clearing everything.
You can uncomment this line any time you want to reset the conversation to start fresh.

In [ ]:
# To reset the conversation history:
# recipe_conversation.clear()

Why would you need to reset memory?

* To start a fresh conversation with no previous context
* To resolve issues when the conversation has gone off track
* To clear sensitive information that was shared earlier
* To test how the agent behaves with and without context
* To simulate a new user session


When to use it:

* During development and testing
* When implementing session management in production systems
* Between different user interactions
* When the context window is getting too large (most LLMs have token limits)

### 2.6 Testing the Recipe Agent

Let's create an interactive experience where you can try out the recipe agent with your own queries.

#### Understanding the Different Query Modes

There are two different ways to interact with the recipe agent:

**Single Query Mode** and **Interactive Recipe Assistant**


**Key Difference: Context Memory**
While both modes connect to the same underlying agent, the interactive mode has the potential to maintain context between questions. For example, if you ask "What can I make with avocado?" and then follow up with "What are the ingredients?", the agent in interactive mode might remember you were discussing avocado recipes. In single query mode, each question stands alone without this context.

Try both modes to see how they differ in user experience and to understand the importance of conversation design in agent applications!

#### What Can You Ask?

The recipe agent can answer questions like:
* **Finding recipes by ingredients**: "What can I make with avocado and milk?"
* **Getting ingredients**: "What ingredients do I need for pasta primavera?"
* **Nutritional information**: "How many calories are in chicken curry?"
* **Dietary information**: "Is banana smoothie vegetarian?"

The agent uses the tool functions we defined earlier to search our recipe database and provide helpful responses. Feel free to experiment with different questions to see how the agent processes your requests and determines which tools to use.


In [ ]:
# Helper function to ensure we're in a properly initialized asyncio environment in Colab
async def run_in_colab(coroutine):
    """Helper to run async functions in Google Colab"""
    try:
        import nest_asyncio
        nest_asyncio.apply()
    except ImportError:
        !pip install nest_asyncio -q
        import nest_asyncio
        nest_asyncio.apply()

    return await coroutine

### **Single query mode**
* This mode allows you to ask just one question at a time
* Each time you run the cell, you'll be prompted for a new question
* The agent doesn't maintain memory between different questions
* Best for: Testing specific question types or demonstrating how the agent handles different queries
* Use this when you want to focus on understanding how a particular type of query works


In [ ]:
# single query mode:
async def try_recipe_query():
    question = input("Enter your recipe question: ")
    await run_in_colab(recipe_query(question))


await run_in_colab(try_recipe_query())


Enter your recipe question: What are the ingredients for pasta primavera?


**Recipe Assistant:** The ingredients for Pasta Primavera are:

- Pasta
- Zucchini
- Bell pepper
- Tomato
- Olive oil
- Garlic

### **Interactive recipe query function**
* This mode provides a continuous conversation experience
* You can ask multiple questions in succession without rerunning cells
* **Special Commands:**
  * Type `examples` to see sample questions you can ask
  * Type `exit` to end the conversation and quit the assistant
* The assistant maintains the conversation flow and remembers previous interactions
* Best for: Exploring the agent's capabilities or simulating a real user interaction
* Use this when you want to experience how an agent would work in a production environment




In [ ]:
# Interactive recipe query function

async def interactive_recipe_assistant():
    """Interactive session with the recipe agent"""
    print("🍳 Welcome to the Recipe Assistant! 🍳")
    print("Ask me about recipes, ingredients, or nutritional information.")
    print("Type 'examples' to see sample questions or 'exit' to quit.")

    examples = [
        "What can I make with avocado and bread?",
        "What are the ingredients for pasta primavera?",
        "Is banana smoothie vegetarian?",
        "What's the nutritional info for chicken curry?",
        "Show me vegetarian recipes"
    ]

    while True:
        user_input = input("\nYour question: ")

        if user_input.lower() == 'exit':
            print("Thanks for using the Recipe Assistant. Goodbye!")
            break

        if user_input.lower() == 'examples':
            print("\nHere are some example questions you can ask:")
            for i, example in enumerate(examples, 1):
                print(f"{i}. {example}")
            continue

        print("\nThinking...\n")
        response = await recipe_workflow.run(user_msg=user_input)
        print(f"🍽️ {response}")

# Run the interactive assistant
await run_in_colab(interactive_recipe_assistant())

🍳 Welcome to the Recipe Assistant! 🍳
Ask me about recipes, ingredients, or nutritional information.
Type 'examples' to see sample questions or 'exit' to quit.

Your question: What can I make with avocado and bread?

Thinking...

🍽️ You can make **Avocado Toast** with avocado and bread. Here are the details:

### Ingredients:
- Bread
- Avocado
- Lemon juice
- Salt
- Pepper

### Nutritional Information (per serving):
- **Calories:** 300
- **Protein:** 5g
- **Fat:** 20g

### Recipe Type:
- Vegetarian

Enjoy your avocado toast!

Your question: What are the ingredients for pasta primavera?

Thinking...

🍽️ The ingredients for Pasta Primavera are:

- Pasta
- Zucchini
- Bell pepper
- Tomato
- Olive oil
- Garlic

Your question: What's the nutritional info for chicken curry?

Thinking...

🍽️ The nutritional information for chicken curry is as follows:

- **Calories:** 520
- **Protein:** 35g
- **Fat:** 22g

Your question: Show me vegetarian recipes

Thinking...

🍽️ It seems that I couldn't find 

#### Understanding What's Happening Behind the Scenes

When you ask the recipe agent a question, here's what happens:

1. The agent receives your natural language query
2. It analyzes what you're asking for
3. It selects the appropriate tool function:
   - `find_recipe_by_ingredients()` when you ask what you can make with certain ingredients
   - `get_ingredients()` when you ask for recipe ingredients
   - `get_nutrition_info()` when you ask about calories or nutritional content
   - `get_recipe_type()` when you ask if a recipe is vegetarian or not
4. It calls that function with the appropriate parameters
5. It formats the response in a helpful, human-readable way

This process demonstrates the power of combining large language models with specific tool functions. The LLM handles the language understanding and generation, while the tools handle the specific data retrieval tasks.

Try asking some questions and think about which tool is being used each time!

## 3. Financial Assistant Agent

In this section, we'll build a different type of agent that combines financial data access with mathematical operations. This demonstrates how agents can integrate multiple capabilities to perform more complex tasks.

### 3.1 Setting Up the Financial Tools

Our financial assistant will need two types of tools:
1. Tools to retrieve stock market data
2. Tools to perform calculations on that data

For the financial data, we'll use Yahoo Finance, which provides real-time stock information. For calculations, we'll create basic math functions that our agent can use to analyze the data.

In [ ]:
# Import Yahoo Finance tools
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec

# Define mathematical tool functions
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

def subtract(a: float, b: float) -> float:
    """Subtract b from a and return the difference"""
    return a - b

def divide(a: float, b: float) -> float:
    """Divide a by b and return the quotient. Requires b != 0"""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

#### Understanding the Financial Tools

Our Financial Assistant uses two categories of tools:

1. **Yahoo Finance Tools**:
   - Provided by the `YahooFinanceToolSpec` class from LlamaIndex
   - Allows access to real-time and historical stock data
   - Can retrieve information like stock prices, company information, and financial metrics
   - These tools connect to external data sources via APIs

2. **Mathematical Operation Tools**:
   - Custom-built functions for basic arithmetic
   - Allow the agent to perform calculations on the financial data
   - Each follows the same pattern: take numeric inputs and return a calculated result
   - Include proper error handling (like preventing division by zero)

**Yahoo Finance Capabilities:**
When we convert the Yahoo Finance tool specification to a list using `.to_tool_list()`, we get access to functions like:
- Getting current stock prices
- Retrieving company information
- Accessing financial metrics like P/E ratios
- Checking historical price data

These tools demonstrate an important principle: agents become more powerful when they can access external data sources and perform operations on that data.

### 3.2 Creating the Financial Agent Workflow

Now we'll combine the Yahoo Finance tools with our math functions to create a financial assistant that can both retrieve data and perform calculations on it. This combination makes our agent much more useful than one that could only do either task alone.

In [ ]:
# Get Yahoo Finance tools and add our custom math functions
finance_tools = YahooFinanceToolSpec().to_tool_list()
finance_tools.extend([multiply, add, subtract, divide])

# Create the finance agent workflow
finance_workflow = AgentWorkflow.from_tools_or_functions(
    finance_tools,
    llm=llm,
    system_prompt="You are a financial assistant that can retrieve stock information and perform calculations. Always round dollar values to 2 decimal places."
)

#### The Financial Agent Architecture

Let's analyze what we've done to create our financial assistant:

1. **Tool Integration**:
   - We first converted the Yahoo Finance tool specification into a list of usable tools
   - We then extended this list with our custom math functions
   - This creates a comprehensive toolset that combines data retrieval and computation

2. **Workflow Creation**:
   - We use the same `AgentWorkflow.from_tools_or_functions()` method as before
   - We pass our combined list of tools to make them available to the agent
   - We reuse the same LLM instance we created earlier

3. **System Prompt Design**:
   - The system prompt is specifically tailored for financial assistance
   - It instructs the agent to retrieve stock information and perform calculations
   - It includes a formatting instruction (rounding dollar values) for consistency
   - This demonstrates how the system prompt shapes agent behavior for different use cases

The power of this approach is that we can easily create specialized agents by:
1. Selecting appropriate tools for the domain
2. Writing a tailored system prompt
3. Using the same underlying LLM and workflow architecture

This modularity is a key advantage of tool-based agents compared to pure LLM applications.

### 3.3 Interacting with the Financial Agent

Similar to our recipe agent, we'll create a function to query our financial agent and display the results nicely.

In [ ]:
# Function to query the finance agent
async def finance_query(question):
    """Send a single query to the financial agent and display the response"""
    response = await finance_workflow.run(user_msg=question)
    display(Markdown(f"**Financial Assistant:** {response}"))
    return response

#### Query Function Design

Our `finance_query` function follows the same pattern as the recipe query function:

1. It's defined as an `async` function, which allows for non-blocking API calls
2. It takes a question as input and passes it to the financial agent workflow
3. It formats the response with Markdown for better readability
4. It returns the raw response for potential further use

The similarity between the two query functions demonstrates an important concept in agent design: once you have the basic architecture in place, adding new agents with different capabilities follows a consistent pattern. This modularity makes it easy to build multiple specialized agents.

While the query function looks the same, the underlying agent's behavior will be very different because:
- It has access to different tools (Yahoo Finance + math functions)
- It operates under a different system prompt (financial assistant vs. cooking assistant)
- It will recognize and respond to finance-specific questions

### 3.4 Adding Conversation Memory to the Financial Agent

Just like with our recipe agent, we'll add conversation memory to the financial agent to enable multi-turn interactions and maintain context throughout the conversation.

In [ ]:
# Create a conversation memory for the financial agent
finance_conversation = []

async def finance_chat_with_memory():
    """Interactive chat with the financial agent that maintains conversation context"""
    print("💰 Financial Assistant Chat! Type 'exit' to quit.")

    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            print("👋 Goodbye!")
            break

        # Add to conversation context
        finance_conversation.append(f"User: {user_input}")
        context = "\n".join(finance_conversation)

        try:
            # Get response
            response = await finance_workflow.run(user_msg=context)
            print(f"💰 Financial Assistant: {response}\n")

            # Add to conversation context
            finance_conversation.append(f"Assistant: {response}")
        except Exception as e:
            print(f"Error: {str(e)}")


In [ ]:
# To reset the conversation history:
#finance_conversation.clear()

#### Conversation Memory for Finance Queries

The conversation memory implementation for our financial assistant mirrors the one we created for the recipe agent:

1. We create a list (`finance_conversation`) to store the conversation history
2. Our chat function appends each user question and agent response to this history
3. When processing a new query, we provide the entire conversation as context

This approach is particularly valuable for financial discussions, which often involve:

- **Follow-up questions**: "What about Microsoft?" after asking about Apple
- **Comparative analyses**: Asking about relationships between previously mentioned stocks
- **Complex scenarios**: Building up a hypothetical investment scenario across multiple interactions

**Example Financial Conversation Flow**:
1. User asks about Apple's current stock price
2. Agent provides the price
3. User asks about the P/E ratio
4. Agent answers, using the context that we're still discussing Apple
5. User asks "How does that compare to Microsoft?"
6. Agent understands "that" refers to the P/E ratio, and the comparison is to Apple

Without conversation memory, step 5 would require the user to explicitly state "How does Apple's P/E ratio compare to Microsoft's P/E ratio?" This explicit repetition makes conversations feel unnatural and tedious.

As with the recipe agent, you can clear the conversation history at any time by uncommenting and running the `finance_conversation.clear()` line.

### 3.5 Testing the Financial Agent

Let's test our financial agent with some example queries that demonstrate its capabilities with both stock data retrieval and calculations.

In [ ]:
# Finance Agent Examples
print("\n=== Financial Agent Examples ===")
await finance_query("What is the current price of Apple stock?")
await finance_query("What is the current price of Microsoft stock?")
await finance_query("What would be the value of 10 shares of Tesla?")




=== Financial Agent Examples ===


**Financial Assistant:** The current price of Apple stock (AAPL) is $217.90.

**Financial Assistant:** The current price of Microsoft stock (MSFT) is $378.80.

**Financial Assistant:** The value of 10 shares of Tesla (TSLA) at the current price of $263.55 per share would be **$2635.50**.

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The value of 10 shares of Tesla (TSLA) at the current price of $263.55 per share would be **$2635.50**.')]), tool_calls=[ToolCallResult(tool_name='stock_basic_info', tool_kwargs={'ticker': 'TSLA'}, tool_id='call_EmBpbhlIihRXuj0eXtN0VRys', tool_output=ToolOutput(content="Info: \n{'address1': '1 Tesla Road', 'city': 'Austin', 'state': 'TX', 'zip': '78725', 'country': 'United States', 'phone': '512 516 8177', 'website': 'https://www.tesla.com', 'industry': 'Auto Manufacturers', 'industryKey': 'auto-manufacturers', 'industryDisp': 'Auto Manufacturers', 'sector': 'Consumer Cyclical', 'sectorKey': 'consumer-cyclical', 'sectorDisp': 'Consumer Cyclical', 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The co

#### Note About Finance Agent Output

When working with the Yahoo Finance tools in LlamaIndex, you might notice that the response includes both:
1. The human-readable answer (e.g., "The current price of Apple stock is $217.90")
2. A detailed AgentOutput object with all the raw data and tool call information

This is normal behavior and actually very useful for debugging and advanced use cases. For educational purposes, we'll focus on the human-readable part, but it's good to know that the full data is available if needed.

If you want to extract just the human-readable text from the response for display in a real application, you would need to parse the response object structure. For this tutorial, we'll keep it simple and focus on the examples.

In [ ]:
# Finance Agent Examples - Simplified to show just text responses
print("\n=== Financial Agent Examples ===")

# Simple stock price query
print("Example 1: Basic stock price query")
response = await finance_workflow.run(user_msg="What is the current price of Apple stock?")
# Just print the first line for a cleaner display
print(f"Financial Assistant: {str(response).split('AgentOutput')[0].strip()}")

# Different stock for variety
print("\nExample 2: Different stock query")
response = await finance_workflow.run(user_msg="What is the current price of Microsoft stock?")
print(f"Financial Assistant: {str(response).split('AgentOutput')[0].strip()}")

# Simple calculation based on stock price
print("\nExample 3: Stock value calculation")
response = await finance_workflow.run(user_msg="What would be the value of 10 shares of Tesla?")
print(f"Financial Assistant: {str(response).split('AgentOutput')[0].strip()}")


=== Financial Agent Examples ===
Example 1: Basic stock price query
Financial Assistant: The current price of Apple stock (AAPL) is $217.90.

Example 2: Different stock query
Financial Assistant: The current price of Microsoft (MSFT) stock is $378.80.

Example 3: Stock value calculation
Financial Assistant: The value of 10 shares of Tesla (TSLA) is $2,635.50.


### 3.6 Advanced Finance Queries

Now that we've seen basic stock price queries, let's explore more complex operations that combine financial data retrieval with calculations. These examples demonstrate the power of combining tools within a single agent.

In [ ]:
# Advanced Finance Query Examples
print("\n=== Advanced Financial Queries ===")

# Combining stock data with calculations
print("Example 1: Calculating investment return")
response = await finance_workflow.run(user_msg="If I invest $1000 in Amazon stock, how many shares can I buy and what would their value be if the price increases by 10%?")
print(f"Financial Assistant: {str(response).split('AgentOutput')[0].strip()}")

# Multiple stocks comparison
print("\nExample 2: Comparing dividend yields")
response = await finance_workflow.run(user_msg="Which has a higher dividend yield, Microsoft or Apple?")
print(f"Financial Assistant: {str(response).split('AgentOutput')[0].strip()}")

# Financial metrics with calculations
print("\nExample 3: P/E ratio analysis")
response = await finance_workflow.run(user_msg="What would happen to Microsoft's P/E ratio if its price decreased by 15%?")
print(f"Financial Assistant: {str(response).split('AgentOutput')[0].strip()}")


=== Advanced Financial Queries ===
Example 1: Calculating investment return
Financial Assistant: If you invest $1000 in Amazon stock at the current price of $192.72, you can buy approximately **5.19 shares**. 

If the price increases by 10%, the new price would be approximately **$211.99**. The total value of your shares at that price would be approximately **$1,099.95** (5.19 shares × $211.99).

Example 2: Comparing dividend yields
Financial Assistant: The dividend yields for Microsoft and Apple are as follows:

- **Microsoft (MSFT)**: 0.88%
- **Apple (AAPL)**: 0.46%

**Conclusion**: Microsoft has a higher dividend yield compared to Apple.

Example 3: P/E ratio analysis
Financial Assistant: If Microsoft's stock price decreased by 15%, the new price would be approximately $321.98. Given the current earnings per share (EPS) of $12.42, the new P/E ratio would be approximately 25.92. 

This represents a decrease in the P/E ratio compared to the current P/E ratio of about 30.50.


### 3.7 Interactive chat session with follow-up questions



In [ ]:
# to start an interactive chat session:
await finance_chat_with_memory()

💰 Financial Assistant Chat! Type 'exit' to quit.
You: Which has a higher dividend yield, Microsoft or Apple?
💰 Financial Assistant: The dividend yields for Microsoft and Apple are as follows:

- **Microsoft (MSFT)**: Dividend Yield = **0.88%**
- **Apple (AAPL)**: Dividend Yield = **0.46%**

Therefore, Microsoft has a higher dividend yield compared to Apple.

You: If I invest $1000 in Amazon stock, how many shares can I buy and what would their value be if the price increases by 10%?
💰 Financial Assistant: If you invest $1000 in Amazon stock at the current price of $192.72, you can buy approximately **5 shares** (since you can't purchase a fraction of a share).

If the price increases by 10%, the new price would be approximately **$211.99** per share. Therefore, the total value of your investment would be:

- **5 shares x $211.99 = $1059.95**. 

So, your investment would be worth **$1059.95** if the price increases by 10%.

You: exit
👋 Goodbye!


### 3.8 Financial Data Analysis Examples

Let's explore some examples of how our financial agent can be used for data analysis tasks beyond simple queries:

1. **Comparative Analysis**: Comparing metrics across companies
2. **Portfolio Evaluation**: Calculating the value and performance of multiple stocks
3. **Trend Analysis**: Looking at how metrics change under different conditions

These capabilities demonstrate how agents can support complex decision-making processes by combining data retrieval with analytical operations.

### 3.9 Using the Financial Agent: Best Practices

When working with the financial agent, keep these tips in mind:

1. **Be specific with tickers**: Use stock tickers (AAPL, MSFT, GOOGL) for more reliable results
2. **Combine queries and calculations**: The agent can both retrieve data and perform calculations on it
3. **Ask about relationships**: The agent can compare stocks or metrics across companies
4. **Start simple**: Begin with basic queries before asking complex multi-step questions
5. **Use conversation memory**: For related follow-up questions, use the chat with memory function
6. **Consider data freshness**: Stock prices and financial metrics change frequently, so results will vary

The financial agent demonstrates a key advantage of the agent-based approach: combining specialized tools (Yahoo Finance data retrieval) with general capabilities (mathematical operations) in a seamless experience.